## Hippopt planner
This example, load a basic robot model (i.e. composed only of basic shapes), modifies the links of such a robot model by elongating the legs, plans a forward walking trajecotry using Hippopt walking planner  

In [ ]:
# Comodo import
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.hippoptWalkingPlanner.hippoptWalkingPlanner  import HippoptWalkingPlanner
from comodo.hippoptWalkingPlanner.hippoptWalkingPlannerParameterTuning import HippoptWalkingParameterTuning

In [ ]:
# General  import 
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request

In [ ]:
# Getting stickbot urdf file and convert it to string 
urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
url = 'https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf'
urllib.request.urlretrieve(url, urdf_robot_file.name)
# Load the URDF file
tree = ET.parse(urdf_robot_file.name)
root = tree.getroot()

# Convert the XML tree to a string
robot_urdf_string_original = ET.tostring(root)

create_urdf_instance = createUrdf(
    original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
)

In [ ]:
# Define parametric links and controlled joints  
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
]

In [ ]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters 
Im_arms = 1e-3*np.ones(4) # from 0-4
Im_legs = 1e-3*np.ones(6) # from 5-10
kv_arms = 0.001*np.ones(4) # from 11-14
kv_legs = 0.001*np.ones(6) #from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [ ]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()
robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()
robot_model_init.set_initial_position(s_init=s_des, w_H_b_init=H_b, xyz_rpy_init=xyz_rpy)
robot_model_init.set_foot_corner(np.asarray([0.1, 0.05, 0.0]),np.asarray([0.1, -0.05, 0.0]),np.asarray([-0.1, -0.05, 0.0]),np.asarray([-0.1, 0.05, 0.0]))

In [ ]:
hippopt_planner = HippoptWalkingPlanner(robot_model_init)
hippopt_parameters = HippoptWalkingParameterTuning()
hippopt_parameters.step_length = 0.2
hippopt_planner.initialize_planner(hippopt_parameters)
hippopt_planner.visualizer_init()
hippopt_planner.visualize_state(hippopt_planner.guess)

In [ ]:
warm_start = hippopt_planner.plan_trajectory()
hippopt_planner.visualize_state(hippopt_planner.humanoid_states)
# print(hippopt_planner.humanoid_states)

In [ ]:
hippopt_planner.warm_start_opt(warm_start)
hippopt_planner.plan_trajectory()
hippopt_planner.visualize_state(hippopt_planner.humanoid_states)

In [ ]:
#  hippopt_planner.warm_start_opt(warm_start)
#  hippopt_planner.plan_trajectory()
hippopt_planner.visualize_state(hippopt_planner.humanoid_states)

In [ ]:

from comodo.centroidalMPC.footPositionPlanner import FootPositionPlanner
foot_pos_planner = FootPositionPlanner(robot_model_init, 0.1,0.6)
foot_pos_planner.plot_feet_position(hippopt_planner.contact_phase_list_left, hippopt_planner.contact_phase_list_right)
i = 0 
for item in hippopt_planner.contact_phase_list_left:
    print(i)
    i +=1
    # print(item)

In [ ]:
import matplotlib.pyplot as plt
frequency_ms = 100
dT_in_seconds = frequency_ms / 1000
com_x = []
com_y = []
com_z= []
time_plot = []
print(len(hippopt_planner.com_traj))
for i in range(len(hippopt_planner.com_traj)):
    com_i = hippopt_planner.com_traj[i]
    com_x.append(com_i[0])
    com_y.append(com_i[1])
    com_z.append(com_i[2])
    time_plot.append(i * dT_in_seconds)

plt.figure()
plt.plot(time_plot, com_x, label = "x")
plt.plot(time_plot, com_y, label = "y")
plt.plot(time_plot, com_z, label = "z")
plt.legend()
plt.title("CoM", fontsize="60")
plt.xlabel("t [sec]", fontsize="40")
plt.ylabel(" [m]", fontsize="40")
plt.show()



In [ ]:
initial_state = hippopt_planner.initial_state
s_des = initial_state.kinematics.joints.positions
xyz = initial_state.kinematics.base.position
xyzw = initial_state.kinematics.base.quaternion_xyzw 
wxyz_xyz = np.asarray([xyzw[3],xyzw[0],xyzw[1] ,xyzw[2], xyz[0], xyz[1], xyz[2]])
from_quat_to_matrix = robot_model_init.from_quaternion_to_matrix()
H_base = from_quat_to_matrix(wxyz_xyz)
xyz_rpy = robot_model_init.matrix_to_rpy(H_base)


In [ ]:
# Comodo import
from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
from comodo.TSIDController.TSIDController import TSIDController

In [ ]:
import matplotlib.pyplot as plt

force_left = [] 
time_plot = []
for i, point in enumerate(hippopt_planner.left_contact_points):
        current_in_contact  = True
        total_force_z = 0.0
        for item in point:
            total_force_z += item.f[2]   
        force_left.append(total_force_z)
        time_plot.append(i*hippopt_planner.time_step)

plt.figure()
plt.plot(time_plot, force_left, label = "x")
plt.legend()
plt.title("Hippopt normal force z  ", fontsize="60")
plt.xlabel("t [sec]", fontsize="40")
plt.ylabel(" [Nm]", fontsize="40")


forces_right = []
for i, point in enumerate(hippopt_planner.right_contact_points):
        current_in_contact  = True
        total_force_z = 0.0
        for item in point:
            total_force_z += item.f[2]   
        forces_right.append(total_force_z)
plt.figure()
plt.plot(time_plot, forces_right, label = "x")
plt.legend()
plt.title("Hippopt normal force z  ", fontsize="60")
plt.xlabel("t [sec]", fontsize="40")
plt.ylabel(" [Nm]", fontsize="40")
print(min(forces_right))
print(min(force_left))


In [ ]:
# Define simulator and set initial position
mujoco_instance = MujocoSimulator()
mujoco_instance.load_model(
    robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im
)
s, ds, tau = mujoco_instance.get_state()
t = mujoco_instance.get_simulation_time()
H_b = mujoco_instance.get_base()
w_b = mujoco_instance.get_base_velocity()
mujoco_instance.set_visualize_robot_flag(True)

In [ ]:
# Define the controller parameters  and instantiate the controller
# Controller Parameters
tsid_parameter = TSIDParameterTuning()
mpc_parameters = MPCParameterTuning()
mpc_parameters.com_weight = np.asarray([10,10,1000])

# TSID Instance
TSID_controller_instance = TSIDController(frequency=0.01, robot_model=robot_model_init)
TSID_controller_instance.define_tasks(tsid_parameter)
TSID_controller_instance.set_state_with_base(s, ds, H_b, w_b, t)

# MPC Instance
step_lenght = 0.1
mpc = CentroidalMPC(robot_model=robot_model_init, step_length=step_lenght)
mpc.intialize_mpc(mpc_parameters=mpc_parameters)

# Set desired quantities
# mpc.set_reference_mpc_ext(hippopt_planner.com_traj, hippopt_planner.ang_mom_traj,hippopt_planner.contact_phase_list)
print(len(hippopt_planner.com_traj))
# TSID_controller_instance.compute_com_position()
# mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())
# Set desired quantities
# mpc.configure(s_init=s_des, H_b_init=H_b)
mpc.configure_with_feet_position(hippopt_planner.contact_phase_list_left, hippopt_planner.contact_phase_list_right)
# TSID_controller_instance.compute_com_position()
# mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())
mpc.com_traj = hippopt_planner.com_traj
mpc.angular_mom_trak = hippopt_planner.ang_mom_traj
# Set initial robot state  and plan trajectories
mujoco_instance.step(1)

# Reading the state
s, ds, tau = mujoco_instance.get_state()
H_b = mujoco_instance.get_base()
w_b = mujoco_instance.get_base_velocity()
t = mujoco_instance.get_simulation_time()

# MPC
mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc.initialize_centroidal_integrator(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
mpc_output = mpc.plan_trajectory()

In [ ]:
# Set loop variables
TIME_TH = 3.1

# Define number of steps
n_step = int(
    TSID_controller_instance.frequency / mujoco_instance.get_simulation_frequency()
)
n_step_mpc_tsid = int(mpc.get_frequency_seconds() / TSID_controller_instance.frequency)
print(n_step_mpc_tsid)
counter = 0
mpc_success = True
energy_tot = 0.0
succeded_controller = True

com_x = []
com_y = []
com_z= []
time_plot = []
forces_left_x = []
forces_left_y = []
forces_left_z = []

forces_right_x = []
forces_right_y = []
forces_right_z = []

In [ ]:
# Simulation-control loop
i = 0
t = 0 
while t < TIME_TH:
    # Reading robot state from simulator
    t += 0.01
    print(t)
    s, ds, tau = mujoco_instance.get_state()
    energy_i = np.linalg.norm(tau)
    H_b = mujoco_instance.get_base()
    w_b = mujoco_instance.get_base_velocity()
    # t = mujoco_instance.get_simulation_time()
    # Update TSID
    TSID_controller_instance.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
    
    # MPC plan
    if counter == 0:
        mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
        mpc.update_references()
        mpc_success = mpc.plan_trajectory()
        mpc.contact_planner.advance_swing_foot_planner()
        if not (mpc_success):
            print("MPC failed")
            break

    # # Reading new references
    com, dcom, forces_left, forces_right, ang_mom = mpc.get_references()
    left_foot, right_foot = mpc.contact_planner.get_references_swing_foot_planner()
    com_x.append(com[0])
    com_y.append(com[1])
    com_z.append(com[2])
    print(com)
    forces_right_x.append(forces_right[0])
    forces_right_y.append(forces_right[1])
    forces_right_z.append(forces_right[2])
    forces_left_x.append(forces_left[0])
    forces_left_y.append(forces_left[1])
    forces_left_z.append(forces_left[2])
    time_plot.append(t)
    
    com_ref = np.zeros(3)
    com_ref[:2] = com[:2]
    com_ref[2] = hippopt_planner.com_traj[1][2]
    # Update references TSID
    TSID_controller_instance.update_task_references_mpc(
        com=com_ref,
        dcom=dcom,
        ddcom=np.zeros(3),
        left_foot_desired=left_foot,
        right_foot_desired=right_foot,
        s_desired=np.array(s_des),
        wrenches_left=forces_left,
        wrenches_right=forces_right,
    )
    com_fun = robot_model_init.CoM_position_fun()
    # print("com current", com_fun(H_b,s))
    # print("com traj", hippopt_planner.com_traj[1])
    # print("com mpc", com)
    # print("velocity com", dcom)

    # Run control
    succeded_controller = TSID_controller_instance.run()

    # if not (succeded_controller):
    #     print("Controller failed")
    #     break

    tau = TSID_controller_instance.get_torque()

    # Step the simulator
    mujoco_instance.set_input(tau)
    mujoco_instance.step(n_step=n_step)
    # mujoco_instance.step_with_motors(n_step=n_step, torque=tau)
    counter = counter + 1

    if counter == n_step_mpc_tsid:
        counter = 0

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time_plot, com_x, label = "x")
plt.plot(time_plot, com_y, label = "y")
plt.plot(time_plot, com_z, label = "z")
plt.legend()
plt.title("CoM MPC Output ", fontsize="60")
plt.xlabel("t [sec]", fontsize="40")
plt.ylabel(" [m]", fontsize="40")

plt.figure()
print(len(forces_left_y))
print(len(time_plot))
plt.plot(time_plot, forces_left_x, label = "x")
plt.plot(time_plot, forces_left_y, label = "y")
plt.plot(time_plot, forces_left_z, label = "z")
plt.legend()
plt.title("Forces left MPC Output ", fontsize="60")
plt.xlabel("t [sec]", fontsize="40")
plt.ylabel(" [N]", fontsize="40")

plt.figure()
plt.plot(time_plot, forces_right_x, label = "x")
plt.plot(time_plot, forces_right_y, label = "y")
plt.plot(time_plot, forces_right_z, label = "z")
plt.legend()
plt.title("Forces rigth MPC Output ", fontsize="60")
plt.xlabel("t [sec]", fontsize="40")
plt.ylabel(" [N]", fontsize="40")

plt.show()

print("left forces", forces_right_z)


In [ ]:
print(hippopt_planner.contact_phase_list.__dir__())
print(hippopt_planner.contact_phase_list[0].active_contacts)

In [ ]:
# Closing visualization
mujoco_instance.close_visualization()